In [1]:
import nltk
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import unicodedata2
import math
import string
import tokenize
import sklearn
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import wordpunct_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import sent_tokenize
from sklearn import metrics
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import polynomial_kernel
from sklearn.naive_bayes import MultinomialNB
from string import digits
from xml.dom import minidom
from unidecode import unidecode
from nltk.stem.snowball import SnowballStemmer
from string import punctuation
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.svm import SVC

In [2]:
data=pd.read_csv("Validation.csv")
data.head()

,TweetId,Text,seguridad
0,1185490280251871232,Hollman alcalde de Tweeter 👏👏👏🥇,0.0
1,1235401570143137792,Mentirosa. Ese temple era puro show,0.0
2,1225390928002781184,Sin novedad estacion terreros,0.0
3,1236756113695084546,"Se partió un árbol en la carrera 9A, frente al...",0.0
4,1227697123321352194,Muy poquitos me parecen porque el 99.9% no cum...,0.0


In [3]:
def curielwords(fileName):
    curielw = []
    words=pd.read_csv(fileName)
    for row in words.WORD:
        curielw.append(row)
    return curielw

In [4]:
cwords=curielwords("crimerelatedwords.csv")

In [5]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    for digits in string.digits:
        text = text.replace(digits,'')
    text = text.lower()
    return text

def preprocessing(text):
    text= text.apply(remove_punctuations)
    return text

data.Text=preprocessing(data["Text"])
print(data.Text)

data.head()

0                          hollman alcalde de tweeter 👏👏👏🥇
1                       mentirosa ese temple era puro show
2                            sin novedad estacion terreros
3        se partió un árbol en la carrera a frente al é...
4        muy poquitos me parecen porque el  no cumple c...
                               ...                        
12963    la muerte le llega a todo el mundo ud es vadal...
12964    sres flip si algo se va a hacer de verdad no s...
12965    gusgomez erikafontalvo citytv  eltiempo elespe...
12966             vienen a traer el virus que los expulsen
12967    sugiero descanzar hoy me toco duro adiós pues ...
Name: Text, Length: 12968, dtype: object


,TweetId,Text,seguridad
0,1185490280251871232,hollman alcalde de tweeter 👏👏👏🥇,0.0
1,1235401570143137792,mentirosa ese temple era puro show,0.0
2,1225390928002781184,sin novedad estacion terreros,0.0
3,1236756113695084546,se partió un árbol en la carrera a frente al é...,0.0
4,1227697123321352194,muy poquitos me parecen porque el no cumple c...,0.0


In [15]:
for row in data.Text:
    words=row.split()
    for word in words:
        if word in cwords:
            tag_seg = 1
            break
        else:
            tag_seg = 0
    print (words, tag_seg)

['Hollman', 'alcalde', 'de', 'Tweeter', '👏👏👏🥇'] 0
['Mentirosa.', 'Ese', 'temple', 'era', 'puro', 'show'] 0
['Sin', 'novedad', 'estacion', 'terreros'] 0
['Se', 'partió', 'un', 'árbol', 'en', 'la', 'carrera', '9A,', 'frente', 'al', 'Éxito', 'de', 'la', 'calle', '134', 'en', 'Bogotá', '@PoliciaBogota', '@BomberosBogota', '@Bogota'] 0
['Muy', 'poquitos', 'me', 'parecen', 'porque', 'el', '99.9%', 'no', 'cumple', 'con', 'lo', 'mínimo', 'para', 'transitar', 'con', 'ese', 'vehículo.', 'Deberían', 'ser', 'más', 'estrictos.', 'Por', 'lo', 'menos', 'reflectivo', 'en', 'la', 'noche', 'obligatorio.'] 0
['ya', 'estan', 'haciendole', 'campaña', 'para', 'seguir', 'legalizandoles', 'los', 'negocios', 'a', 'los', 'narcoterroristas', 'criminales', 'de', 'lesa', 'humanidad', 'de', 'las', 'farc', 'y', 'eln'] 0
['@Noti90Minutos', '@EFEnoticias', '@el_pais', '@elespectador', '@HassNassar', '@DELAESPRIELLAE', '@NoticiasCaracol', '@NoticiasRCN', '@NTN24Zoom', '@CNNEE', '@WSJ', '@RedMasNoticias', '@NoticiasUno'

['Ud.', 'Tiene', 'que', 'prohibir', 'las', 'CAPUCHAS', '¡YÁ!'] 0
['Lo', '"curioso', 'de', 'la', 'jornada":', 'el', 'K23', 'se', 'demora', 'tanto', 'en', 'pasar', 'que,', 'los', 'que', 'ya', 'llevábamos', 'un', 'buen', 'rato', 'esperando,', 'alcanzamos', 'a', 'ver', 'el', 'robo,', 'a', 'la', '@PoliciaBogota', 'corriendo', 'detrás', 'de', 'los', 'ladrones', 'y', 'devolviendo', 'a', 'uno', 'a', 'la', 'Estación', 'y', 'a', 'esperar', 'otro', 'rato', 'más', 'y', 'no', 'pasaba😂😂', '@TransMilenio'] 1
['¡', 'Ya!', 'Pasen', 'la', 'pagina', 'hay', 'miles', 'de', 'problemas', 'en', 'Colombia', 'como', 'para', 'seguir', 'con', 'el', 'beso', 'de', 'una', 'pareja', 'que', 'expresan', 'su', 'amor..', 'El', 'Cauca', 'por', 'ejemplo', 'no.!'] 0
['Pero', 'que', 'va', 'desde', 'soacha', 'a', 'chia', 'y', 'vuelve', 'o', 'cuántos', 'coje', 'al', 'día', '🤣'] 0
['El', 'asunto', 'es', 'que', 'en', 'ese', 'polígono', 'que', 'definía', 'el', 'POT', 'hay', 'casas', 'de', 'familia', 'y', 'asobares', 'lo', 'niega.

['Noticias', 'sin', 'sentido?', 'Cada', 'vez', 'más', 'parecidos', 'a', '@NoticiasRCN', 'y', '@NoticiasCaracol', 'periodismo', 'basura'] 0
['@FunemeAlejandro', '@Vicenteaojeda', '@ledoleon', '@cesar_alejo'] 0
['Ya', 'no', 'les', 'basta', 'con', 'destruir', 'el', 'centro,', 'ahora', 'le', 'toca', 'al', 'norte,', 'incluye', 'destrucción', 'de', 'transmilenio,', 'de', 'bien', 'público,', 'robo', 'y', 'ataque', 'a', 'periodistas,', '¿qué', 'sigue?', '#LosEstudiantesMarchanPor', 'seguirse', 'ganando', 'el', 'repudio', 'de', 'la', 'sociedad.'] 1
['Lo', 'que', 'si', 'parece', 'cierto', 'es', 'que', 'la', 'inteligencia', 'de', 'la', '@PoliciaBogota,', 'su', 'capacidad', 'operativa', 'y', 'preventiva', 'en', 'materia', 'de', 'manejo', 'del', 'orden', 'público,', 'resulta', 'de', 'gran', 'ineptitud.', 'El', 'P.@IvanDuque', 'y', 'el', 'A.@EnriquePenalosa', 'deben', 'tomar', 'cartas', 'directas', 'en', 'el', 'asunto.', 'https://t.co/8p5Y3Appum'] 0
['Me', 'van', 'a', 'hacer', 'chillar', 'de', 'orgu

['"Al', 'Secretario', 'de', 'Seguridad', 'no', 'le', 'gusta', 'el', 'tema', 'de', 'la', 'militarización', 'de', 'Bogotá".@EmelRojasC', '@LibresOficial', '@ColJustaLibres', '@RicardoAriasM', '@SeguridadBOG', '@PoliciaBogota', '@CarabinerosCol', '@mindefensa', 'https://t.co/Zp9qwxeW0e'] 0
['Quien', 'ejerce', 'control', 'sobre', 'los', 'supuestos', 'datos', 'que', 'arrojan', 'estas', 'estaciones?', '@Ambientebogota', 'son', 'jueces', 'y', 'parte?', 'Si', 'despues', 'de', '10', 'días', 'de', 'restricción', 'no', 'hay', 'resultados', 'positivos', 'no', 'hay', 'que', 'ser', 'muy', 'inteligente', 'para', 'entender', 'q', 'el', 'problema', 'no', 'son', 'los', 'particulares.', 'INEPTOS,', 'ABUSIVOS!'] 0
['En', 'portal', 'sur', 'también', 'pasa,', 'los', 'perritos', 'se', 'ven', 'en', 'muy', 'mal', 'estado.', 'Pero', 'ellos', 'no', 'permiten', 'ni', 'que', 'uno', 'se', 'acerque.'] 0
['Ojo', 'con', 'esos', 'procedimientos', '@PoliciaColombia', '@DefensoriaCol', '@PGN_COL', '@ONU_derechos', '@Clau

['Brian', 'Osterloh,', 'dir', 'del', 'dto', 'de', 'Tecnología', 'de', 'Albuquerque', '🇺🇸', 'me', 'contó', 'cómo', 'el', 'gobierno', 'le', 'creó', 'una', "'skill'", 'a', 'Alexa', 'para', 'que', 'los', 'ciudadanos,', 'con', 'este', 'asistente', 'virtual,', 'puedan', 'hacer', 'las', 'solicitudes', 'que', 'antes', 'se', 'hacían', 'en', 'la', 'linea', '311', '(como', 'nuestro', '123)', 'Más', 'rápido', 'y', 'eficiente', '🙌👇'] 0
['Que', 'horror', 'sigue', 'siendo', 'una', 'porquería', 'de', 'sistema', 'de', 'transporte', '@EnriquePenalosa'] 0
['"Trabajamos', 'para', 'mejorar', 'las', 'condiciones', 'de', 'seguridad', 'en', 'la', 'localidad', 'de', 'Engativá,', 'en', 'trabajo', 'conjunto', 'con', '@PoliciaBogota', 'hicimos', '15', 'juntas', 'zonales', 'y', 'nuevos', 'frentes', 'de', 'seguridad', 'para', 'fortalecer', 'la', 'vigilancia', 'y', 'mitigar', 'los', 'índices', 'de', 'delincuencia.', '#ImpopularesPeroEficientes✅"'] 1
['Esta', 'más', 'cagada', 'que', 'parapeto', 'de', 'Lora', 'como', 

...

['Para', 'las', 'personas', 'endedudarse'] 0
['#crimenodefensa', '@PoliciaBogota', '@Bogota', 'en', 'la', 'oreja', 'de', 'la', 'calle', '63', 'para', 'tomar', 'la', 'NQS,', 'ahí', 'al', 'lado', 'para', 'ir', 'hacía', 'el', 'Movistar', 'arena', 'debajo', 'del', 'puente', 'vehicular', 'se', 'la', 'pasan', '3', 'ladrones', 'en', 'bicicleta', 'mirando', 'a', 'quien', 'robar', 'peatones', 'o', 'carros', 'con', 'ventanas', 'abiertas,', 'y', 'por', 'ahí', '0', 'policías'] 1
['Que', 'de', 'Simulacro', 'pasó', 'a', 'Fuga.'] 0
['#SubeElTelón', 'Inspirada', 'en', 'el', 'Cañón', 'del', 'Chicamocha,', 'Beatriz', 'González', 'diseñó', 'el', 'telón', 'de', 'boca', 'del', 'Teatro', 'Santander,', 'en', 'Bucaramanga.', 'Inauguración', 'a', 'las', '6:00', 'p.m.', 'Toda', 'la', 'historia', 'en', '#Citynoticias', 'de', 'las', '8', 'p.m', '@Citytv'] 0
['Porque', 'estamos', 'en', 'Colombia!!!', 'El', 'sueldito', 'no', 'le', 'alcanza', 'y', 'debe', 'buscar', 'otro', 'emprendimiento', 'para', 'subsistir....'] 

['Les', 'podemos', 'enviar', 'unas', 'ratas', 'que', 'hay', 'por', 'acá?'] 0
['@CaracolRadio', '@gusgomez1701', '¿o', 'sea', 'que', 'debemos', 'dotar', 'a', 'la', '@PoliciaBogota', '@PoliciaColombia', 'de', 'caucheras', 'y', 'piedritas,', 'como', 'quiere', 'HRW?', '#Yoapoyoalesmad'] 0
['Háblame', 'por', 'instragram', 'o', 'facebook!', 'Fue', 'tenaz', 'Juan', 'Santi'] 0
['Así', 'debería', 'ser.', 'Y', 'que', 'controlen', 'los', 'buses', 'chimeneas,', 'sitp', 'y', 'sitp', 'provisional', 'que', 'son', 'los', 'más', 'infractores', 'con', 'la', 'norma.'] 0
['...Y', 'que', 'la', 'policía', 'en', 'Colombia', 'no', 'quiere', 'a', 'sus', 'ciudadanos.', 'Que', 'nuestra', '@PoliciaBogota', 'sale', 'es', 'a', 'cumplir', 'una', 'cuota.', 'Que', 'no', 'salen', 'a', 'capturar', 'delincuentes', 'sino', 'a', 'pescar', 'a', 'quien', 'se', 'esté', 'comiendo', 'un', 'bocado', 'en', 'la', 'calle.', 'Malditos!'] 1
['Con', 'gusto', 'Lili!...', 'será', 'que', 'no', 'vió', 'tu', 'trino?', '@juliherrerap'] 0
['

['Me', 'encanta', 'esta', 'medida', 'la', 'ciudad', 'es', 'más', 'ágil', 'no', 'se', 'pierde', 'tanto', 'tiempo', 'en', 'trancones', 'y', 'menos', 'stress', '..ya', 'no', 'hay', 'vías', 'para', 'andar'] 0
['Eso', 'q', 'dice', 'no', 'hace', 'sentido.', 'Léalo', 'y', 'dese', 'cuenta'] 0
['viernes', '10:30pm', 'dos', 'ladrones', 'apuñalan', 'a', 'un', 'ciclista', 'en', 'el', 'puente', 'peatonal', 'de', 'la', 'estación', '@TransMilenio', 'Av.', 'Rojas.', '@Citytv', '@CanalCapital', '@Bogota'] 1
['Y', 'quién', 'controla', 'eso,', 'por', 'todo', 'lado', 'hay', 'gente', 'lucrandose', 'con', 'la', 'venta', 'de', 'pasajes', 'en', 'la', '68', 'en', 'la', 'floresta', 'también', 'lo', 'hacen'] 0
['@PoliciaBogota', 'Esto', 'es', 'lo', 'que', 'hace', 'la', 'institucionalidad', 'no', 'sólo', 'en', 'Bogotá,', 'en', 'toda', 'Colombia.'] 0
['Igual', 'esto', 'por', 'la', '30', 'está', 'colapsado,', 'cómo', 'pretenden', 'igualar', 'esto', 'con', 'un', 'metro'] 0
['Buen', 'día,', 'en', 'Medimás', 'EPS', 'd

['#ABCdecretoAmbulancias', 'La', 'coordinación', 'del', 'servicio', 'de', 'ambulancias', 'por', 'parte', 'de', '@SectorSalud', 'permite', 'ahora', 'garantizar', 'mayor', 'seguridad', 'al', 'paciente,', 'evitar', 'que', 'varias', 'ambulancias', 'se', 'desplacen', 'a', 'atender', 'un', 'mismo', 'evento', 'y', 'un', 'uso', 'responsable', 'de', 'los', 'carriles', 'de', '@TransMilenio'] 1
['Una', 'razon', 'más', 'para', 'cerrar', 'el', 'hpta', 'aeropuerto'] 0
['Residentes', 'de', 'Arborizadora', 'baja', 'solicitan', 'la', 'intervención', 'de', 'una', 'de', 'sus', 'vías.', 'Los', 'detalles', 'en', '@Citytv.', 'Haga', 'sus', 'denuncias', 'para', 'que', 'City', 'llegue', 'a', 'su', 'barrio.', 'Escríbanos', 'al', 'correo', 'lauapo@citytv.com.co.', '#ReporteroCiudadano'] 0
['A', 'eso', 'voy.', 'Piden', 'legalización', 'para', 'que?', 'Si', 'así', 'ya', 'nos', 'estamos', 'matando...'] 0
['Hay', 'le', 'hablan', '@AlvaroUribeVel', 'le', 'está', 'colocando', 'competencia', 'los', 'mismos', 'gringos.

['Claro', 'que', 'si,', 'Gustavo', 'Perro', 'nuestro', 'salvador.', 'Usted', 'si', 'que', 'no', 'sabe', 'quién', 'fue', 'Hugo', 'Chávez', 'y', 'me', 'imagino', 'que', 'tampoco', 'se', 'ha', 'enterado', 'de', 'cómo', 'ese', 'tipo', 'destruyó', 'la', 'economía', 'más', 'próspera', 'de', 'Latino', 'America', 'en', 'menos', 'de', '20', 'años,', 'pero', 'bueno', 'siga', 'creyendo', 'lo', 'que', 'quiera'] 0
['Hola', 'genio,', 'es', 'que', 'llegaste', 'a', 'la', 'ciudad', 'con', 'más', 'casos', 'positivos', 'reportados.', 'Así', 'que', 'da', 'igual!', 'Si', 'necesitas', 'información', 'puedes', 'ver', 'tv.'] 0
['Hablamos', 'en', 'varios', 'medios', 'sobre', 'los', 'esquemas', 'de', 'seguridad', 'y', 'el', 'presupuesto', 'de', 'gasolina', 'para', 'los', 'Concejales', 'de', '#Bogotá.', 'Les', 'comparto', 'algunos', 'apartes', 'de', 'estas', 'entrevistas.', '#MiPasiónEsBogotá', '@ConcejoDeBogota', '@NoticiasRCN', '@CanalCapital'] 1
['Yo', 'escuche', 'hace', 'por', 'expertos', 'que', 'el', 'model